# Prerequisites

## Create Delta Table
id, name, dob, email, gender, country, region, city, asset, marital_status

In [0]:
%sql

CREATE TABLE quickstart_catalog.quickstart_schema.users_int(
  id INT,
  name STRING,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
) USING DELTA;

DESCRIBE EXTENDED quickstart_catalog.quickstart_schema.users_int

# Data Ingestion using Copy Into Command - Using Manual Schema Evolution

In [0]:
%sql
COPY INTO quickstart_catalog.quickstart_schema.users_int FROM (
    SELECT
      id::INT,
      name::STRING,
      to_date(dob, 'yyyy-MM-dd') as dob,
      email::STRING,
      gender::STRING,
      country::STRING,
      region::STRING,
      city::STRING,
      asset::INT,
      marital_status::STRING
    FROM '/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/users_staging/'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')


In [0]:
%sql
SELECT count(*) FROM quickstart_catalog.quickstart_schema.users_int


## Alter table manually

In [0]:
%sql
ALTER TABLE quickstart_catalog.quickstart_schema.users_int ADD COLUMNS(education STRING);
DESCRIBE FORMATTED quickstart_catalog.quickstart_schema.users_int;

## Add Education

In [0]:
%sql
COPY INTO quickstart_catalog.quickstart_schema.users_int FROM (
    SELECT
      id::INT,
      name::STRING,
      to_date(dob, 'yyyy-MM-dd') as dob,
      email::STRING,
      gender::STRING,
      country::STRING,
      region::STRING,
      city::STRING,
      asset::INT,
      marital_status::STRING,
      education:: STRING
    FROM '/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/users_staging/'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')

In [0]:
%sql
SELECT * FROM quickstart_catalog.quickstart_schema.users_int WHERE id > 2501 LIMIT 4;

# Data Ingestion using Copy Into Command - Using Automated Schema Evolution

In [0]:
%sql

DROP TABLE quickstart_catalog.quickstart_schema.users_int;
CREATE TABLE IF NOT EXISTS quickstart_catalog.quickstart_schema.users_int;
DESCRIBE FORMATTED quickstart_catalog.quickstart_schema.users_int;

## Automatically reads new column

In [0]:
%sql
COPY INTO quickstart_catalog.quickstart_schema.users_int
FROM '/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/users_staging_new/'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true', 'mergeSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')

In [0]:
%sql
SELECT * FROM quickstart_catalog.quickstart_schema.users_int LIMIT 4;